# Using the Cochran-Mantel-Haenszel statistic to learn about masculinity in America

FiveThirtyEight, a web site about polling, sports, and statistics, worked with a few other parties to run a survey of men's relation to masculinity in the #meToo era. The discussion is here:
https://fivethirtyeight.com/features/what-do-men-think-it-means-to-be-a-man/ .

It is a short article for a generalist audience, and so mostly discusses each question's responses individually, without many cross-tabulations like "How does the likelihood of self-perceived masculinity differ between married and single?"

But the staff at FiveThirtyEight have graciously and openly posted the data. So we can look at that question, or whether men are more likely to pay on dates when they self-report as high-masculinity or when they want to be _perceived_ as masculine, or how reporting as high-masculinity changes the odds of having kids.

This page supplements <a href="https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3197362">An Analysis of U.S. Domestic Migration via Subset-stable Measures of Administrative Data</a>, a paper analyzing 82 million moves made by members of the U.S. formal economy, 2001-2015. To properly do such an undertaking, the paper develops the Cochran-Mantel-Haenszel (CMH) statistic to answer questions about the relationship between factors. For example,
the likelihood of moving for people who are married is lower than the likelihood that singles move, but the married are also more likely to have kids and a mortgage. Using the CMH statistic to control for other characteristics, we find that marrieds are much more likely to move than singles.

This page will:

* Ask lots of interesting questions about the relationship between masculinity and other opinions and factors.
* Give you a tutorial introduction to the CMH statistic and the `cmh.py` package.

Because this is a show-your-work document, some segments get into finer technical details than others, and will be marked as such. That said, let's get on with the analysis.

##  First, download the data and prep some variables.

The survey has about 1,000 responses, which is enough to get reasonalby reliable statistics using a few controls. Many of your questions about the survey questions may be answered by the full instrument itself, at https://github.com/fivethirtyeight/data/blob/master/masculinity-survey/masculinity-survey.pdf.

_Technical_: The next three cells provide and run the functions to do so, producing a Pandas data frame named `d`.

In [1]:
from urllib.request import urlopen
import pandas as pd  
                                            
Data_URL = "https://github.com/fivethirtyeight/data/blob/master/masculinity-survey/raw-responses.csv?raw=true" 
 
def get_data():    
    """Download a copy of the survey if we don't already have it. Return a data frame with the observations."""
    try:
        return pd.read_csv(open("survey.csv", 'rb'))
    except FileNotFoundError:                     
        in_csv = urlopen(Data_URL).read().decode('utf-8')
        f = open("survey.csv", 'w')                      
        for data in in_csv:                              
            f.write(data)
        return pd.read_csv(open("survey.csv", 'rb'))

In [2]:
def prep_data(d):
    """Give the columns readable names, then recode some of the codes to numeric values.
    Some of these are unused below, but may be good points for further exploration."""

    d.rename(columns= {
        'q0001': 'masc_self_rate',        
        'q0002': 'perception_importance', 
        'q0005': 'societal_pressure',
        'q0010_0007': 'advantage_at_work',
        'q0011_0004': 'disadvantage_at_work',
        'q0018': 'pays_on_dates',         
        'q0021': 'concerns_about_too_far',
        'q0024': 'married',
        'q0025': 'kids',  
        'q0026': 'sexual_orientation',  
        'q0027': 'age',   
        'q0028': 'race',
        'q0029': 'schooling'
        }, inplace = True)

    """Encode text values to numeric. For control variables this is nice but not necessary."""
    likert_λ = (lambda x:
        1 if (x.startswith('Very')) else 
        0 if (x.startswith('Somewhat') or x.startswith('Not very') or x.startswith('Not at all'))
        else -1)
    none_λ = lambda x: 0 if str(x).startswith("None") else 1
    
    """Encode the masculinity self-rating, and listwise delete those that did not reply to the key questions."""
    d.loc[:,"masc_self_rate"] = d["masc_self_rate"].apply(likert_λ)
    d.loc[:, "perception_importance"] = d["perception_importance"].apply(likert_λ)
    d = d.loc[d["masc_self_rate"]>=0]
    d = d.loc[d["perception_importance"]>=0]

    d.loc[:, "advantage_at_work"] = d["advantage_at_work"].apply(none_λ)
    d.loc[:, "disadvantage_at_work"] = d["disadvantage_at_work"].apply(none_λ)
    d.loc[:, "sexual_orientation"] = d["sexual_orientation"].apply(lambda x: 0 if x == "Straight" else 1)
    d.loc[:, "married"] = d["married"].apply(lambda x: 1 if x == "Married" else 0)
    d.loc[:, "age3"] = d["age3"].apply(lambda x: 0 if x == "18 - 34" else 1 if x == "35 - 64" else 2)
    return d

In [3]:
# Go
d = prep_data(get_data())

## Now for some risk ratios

We are now ready to look at how variables relate.

For example, how does being married relate to the chance of self-describing as "very masculine"? The first way to answer this is a simple crosstab, as shown below. The survey has weighted results to adjust the sample of respondents to more closely match the population at large, so it is preferable to get an aggregate total (`agg`) using the sum of weights, not simple observation counts.

As per the encodings above, unmarried=0, married=1, report high masculinity=1, not=0.

In [4]:
d.groupby(["married","masc_self_rate"]).agg({"weight": sum})

weight
married masc_self_rate            
0       0               409.673330
        1               243.399440
1       0               255.567705
        1               238.792309

The Python output is a little awkwardly formatted, so here is the table of possibilities reformatted in static text, including married=no and high masculinity self-rate=yes, married=yes and high masculinity-self-rate=no:

|               |single     |  married  | 
|---------------|-----------:|-----------:|
|  not high  | h$_n$ m$_n$ = 409.67 | h$_n$ m$_y$ = 255.57 |
|  high  | h$_y$ m$_n$ = 243.4 | h$_y$ m$_y$ = 238.79 |


Among the married, the likelihood or _risk_ of self-reporting as very masculine is the count of marrieds who claim high masculinity divided by the total count of all marrieds:

$$\frac{h_y m_y}{(h_y m_y + h_n m_y)}= \frac{238.79}{238.79 + 255.57}=48.3\%$$

There is a similar risk for the unmarried, which will be 35.9%. These are the two risks to compare, and their ratio is 48.3/35.9 = 129.6%, which we can read to mean that the married are 1.296 times as likely to self-rate as high masculinity than the unmarried.

At this basic level, the CMH statistic will give us exactly this:

In [5]:
from cmh import cmh

mar_to_masc = cmh(d, "married", "masc_self_rate", "weight", [])
print(f"The risk of self-reporting as somewhat/very masculine for marrieds relative to singles: {mar_to_masc:.1%}")

The risk of self-reporting as somewhat/very masculine for marrieds relative to singles: 129.6%


The order of arguments to the `cmh` function are "independent" followed by "dependent"; picture an arrow `married → masc_self_rate`.

#### _Technical_: again, with more detail

For those of you here to understand the use of the CMH statistic for controlled experiments, let's step through the calculation was done in this no-control case in a little more detail. The controlled version will be just one step further.

As above, we want the ratio of risk of high self-rate given married versus the risk of same given single. Writing it out is a bit of a mess.

$$ \frac{\frac{h_y m_y}{(h_y m_y + h_n m_y)}}{\frac{h_y m_n}{(h_y m_n + h_n m_n)}} = 
\frac{
\frac{238.79}{238.79 + 255.57}
}{
\frac{243.4}{243.4 + 409.67}
} = \frac{48.3\%}{35.9\%}
=129.6\%$$

The denominator can be flipped to reduce this fraction-of-fractions to a more legible (and more useful) simple fraction:
$$ \frac{(h_y m_y)(h_y m_n + h_n m_n)}{(h_y m_n)(h_y m_y + h_n m_y)} = \frac{238.79 \cdot (243.4 + 409.67)}{243.4 \cdot (238.79 + 255.57)} = 113\%$$
This is more than a convenience: these are the numerators and denominators the CMH will use to aggregate multiple risk ratios below.

We can ask the CMH calculator to be a little more verbose in how it did the math. It will present a table giving the total weight for dependent=yes and independent=yes (`dyiy`), then dependent=yes and independent=no (`dyin`), and so on. These numbers match the crosstab above. The numerator and denominator of the de-compounded fraction above is also given, after scaling both numerator and denominator by total weight (so: $238.79 \cdot (243.4 + 409.67)/1147.43=135.911$ and similarly for denominator).

In [6]:
cmh(d, "married", "masc_self_rate", "weight", [], verbose=True)

Independent: married, dependent: masc_self_rate, controls: ['1']
         dyiy       dyin        dniy       dnin       weight         num  \
1                                                                          
1  238.792309  243.39944  255.567705  409.67333  1147.432783  135.911015   

         den  
1             
1  104.86623  


1.296041776986748

_Technical, on confidence intervals (CIs)_: the ratio of two sums of normalized products of large-$N$ binomial distributions is an $F$ distribution, but, honestly, you could also just bootstrap the confidence intervals. In the situation here, either approach is something of a hack, because we don't know enough about the survey to take into account the assigned sample weights. Nonetheless, here is a simple bootstrapping function in case you'd like to try it on the results below.
Using it on the married → masculine self-rate question, we see this approximation of the 95% CI is entirely above zero with a bottom end circa 106%. Because it is the range of a ratio, some readers may prefer to transform the CI to a log scale.

In [7]:
from numpy.random import choice

def boot(d, dep, indep, weights, controls, **kwargs):
    """Arguments match the definition of the cmh function, to make it easy
       to cut/paste/modify one call into the other"""
    sample_CMHs = []
    for _ in range(500):
        dsub = choice(d.index, size=int(len(d) * 0.90))
        sample_CMHs.append(cmh(d.loc[dsub], dep, indep, weights, controls, **kwargs))
    μ = sum(sample_CMHs)/len(sample_CMHs)
    sample_CMHs.sort()
    print(f"""μ={μ:.3};
95% CI = ({sample_CMHs[int(len(sample_CMHs)/20)]}, {sample_CMHs[int(len(sample_CMHs)*19/20)]})
""")
    
boot(d, "married", "masc_self_rate", "weight", [])

μ=1.3;
95% CI = (1.0586707075502633, 1.6002666441099487)



## Like the real world, the CMH is asymmetric

The CMH statistic gives different answers to the question _does $A$ influence $B$?_ and the question _does $B$ influence $A$?_, meaning it can give more information pertinent to a causality inquiry than symmetric correlation-based measures.

Let's look at how report of masculinity relates to another survey question, "How important is it to you that others see you as masculine?", twice:

perception → masculinity:
How does the chance of reporting as very masculine rise for those who report high importance to being perceived as masculine, relative to those who do not?


masculinity → perception: 
How does the chance of giving high importance to being perceived as masculine rise for those reporting as high masculinity, relative to those who do not?

More about controlling for confounding factors below, but those who self-report as not heterosexual may think about issues about how others perceive their masculinity in a very different manner from how the majority of the population thinks about the same issues. So we'll include a control for sexual orientation.

In [8]:
print(f"""
perception → masculinity self-rate
{cmh(d, "perception_importance", "masc_self_rate", "weight", ["sexual_orientation"]):.1%}

masculinity self-rate → perception
{cmh(d, "masc_self_rate", "perception_importance", "weight", ["sexual_orientation"]):.1%}
""")


perception → masculinity self-rate
219.9%

masculinity self-rate → perception
371.3%



So: men who think it's important that others see them as masculine are 2.19 times as likely to self-report as masculine than comparable men who don't think it's "very important", but in the other direction, men who deem themselves "very masculine" are 3.7 times as likely to think it's important that everybody knows about this than comprable other men. Nobody is surprised that influence goes in both directions, but we now have support for the claim that self-perception drives the desire for public perception more than the desire for public perception drives self-perception.

## Controlling for confounding factors

In the intro, I gave the example of how married people, by simple percents, move less often than singles, but controlling for having kids, a mortgage, and other statuses, the married move more often. The CMH statistic was originally intended for these sorts of controlled experiments.

In the married → high masculinity self-report example above, the controls were empty (the last argument to the calls to `cmh` were `[]`), and the  risk ratio calculated via `cmh` was 113%. Let's control for kids. Now, the CMH calculation generates a table of numerator and denominator calculations just like the one above, but with a different row for every value of the control: one row for everybody with a kid, one row for everybody without one. That is, we're running a separate controlled experiement for every condition.

Then the CMH statistic, the aggregate risk ratio, is the weighted sum of numerators over the weighted sum of denominators for each of these separate experiments. See below for why this is the only sensible way to aggregate the results of the sub-experiments.

In [9]:
print(f"""
Aggregate risk ratio: {cmh(d, "married", "masc_self_rate", "weight", ["kids"], verbose=True):.1%}""")

Independent: married, dependent: masc_self_rate, controls: ['kids']
                    dyiy        dyin        dniy        dnin      weight  \
kids                                                                       
Has children  185.527509  105.982297  193.123395  118.633004  603.266204   
No children    53.264800  136.243520   61.726902  289.995675  541.230897   

                    num        den  
kids                                
Has children  69.077825  66.521698  
No children   41.947985  28.946748  

Aggregate risk ratio: 116.3%


Indeed, the change in risk of self-reporting as somewhat/very masculine given married versus non-married is smaller, shifting from proportionately up 30% above to 16% here. Kids really do have an effect.

If you try the bootstrap function above, you'll see that the rough 95% confidence interval ranges from below to above one, so a frequentist might say "controlling for children, marriage has no effect on masculinity self-report". The rest of us would say "a large part of the effect of marriage on masculinity self-report is due to married people having more kids".

## Stability given subsets

This is a survey of attitudes in the United States, so we have to take race into account. Now our controls are `["race", "kids"]`, and we have a separate calculation for each of (Asian, with kids), (Asian, no kids), (Black, kids), (Black, no kids), and so on down the list.

Controlling for race will have a minimal effect on the marriage-to-masculinity report relationships, but by producing a verbose aggregation table, we see why: the weighed survey is heavily White. The White-to-Asian ratio in the general U.S. population is about ten-to-one, and here is 19-to-one; similarly for other minorities. There are a lot of choices to be made in selecting survey weights, especially in a situation where only one weights column can be provided, and it seems the designers chose weights that focus on other aspects, perhaps age or sexual orientation.

In [10]:
print(f"""
Controlling for race and kids:
{cmh(d, "married", "masc_self_rate", "weight", ["race", "kids"], verbose=True):.1%}

Controlling only for kids:
{cmh(d, "married", "masc_self_rate", "weight", ["kids"]):.1%}
""")

Independent: married, dependent: masc_self_rate, controls: ['race', 'kids']
                             dyiy       dyin        dniy        dnin  \
race     kids                                                          
Asian    Has children    2.130738   3.248746    4.367379    0.794569   
         No children     0.132485  15.607672    2.005175    9.459817   
Black    Has children   21.222513   9.824908   15.526566   33.164907   
         No children     2.025883  29.244354    1.019701   41.357585   
Hispanic Has children   41.951812  23.908793   15.145155   16.145682   
         No children    15.583816  19.067693   12.157977   40.015506   
Other    Has children    8.369172   8.731168    7.983513    7.137852   
         No children     3.066757   4.780218    0.102593    9.994901   
White    Has children  111.853274  60.268682  150.100782   61.389994   
         No children    32.455859  67.543584   46.441456  189.167866   

                           weight        num        den  
r

As a final exercise, let's compare only Whites, everybody, then only non-Whites. Throughout, we'll keep race as a control, though it is irrelevant in the first case with only one race.

In [11]:
print(f"""
{cmh(d.loc[d["race"]=="White"], "married", "masc_self_rate", "weight", ["race","kids"]):.1%} ← only Whites
{cmh(d, "married", "masc_self_rate", "weight", ["race","kids"]):.1%} ← Everybody
{cmh(d.loc[d["race"]!="White"], "married", "masc_self_rate", "weight", ["race","kids"]):.1%} ← only non-Whites
""")


105.7% ← only Whites
119.4% ← Everybody
142.1% ← only non-Whites



That is, if you start with the overall married → high-masculinity self-rate CMH statistic (119.4%), and then I tell you that the person is White, then you will adjust your expectation of the effect of married → masc downward to 105.7%. If I tell you that the person is non-White, then you will adjust your expectations upward to 142.1%.

This is extremely natural. But what if the results looked like this:
```
105.7% ← Everybody
119.4% ← only Whites
142.1% ← only non-Whites
```
In this counterfactual, you would start with a baseline of 105.7%, and then you gather more information about the observation, and then your estimate shifts upward _no matter what new information you have_. It would not be hard to find examples where this happens for regression parameters, simple averages of risk ratios, or other popular statistics.

The paper linked at the top of this notebook refers to the natural ordering as _subset stability_, and proves that the CMH statistic is the _only_ aggregate risk ratio among those admissible that guarantees this behavior.


To summarize the methodological part of this notebook: controlled pseudo-experiments are still the gold standard of scientific inquiry, and the CMH statistic is an excellent option for asking all-else-equal questions about how variables relate. The controls are literal, such as comparing only those married with kids to others married with kids, and we can comfortably drill down into statistics of subgroups because the CMH statistic is subset stable. An appendix to the above paper lists several other manners in which the CMH statistic improves over correlation-based statistics like regression coefficients.

# A few more masculinity stats

But before concluding, let's ask a few more questions of the survey.

Which has better support in the data: men are more likely to pay on dates because they perceive themselves as masculine, or because they want _others_ to perceive them as masculine? There is a little more support for the self-perception, and in fact, concern with others' perception has close to no influence on whether men pay.

_Technical_: The data prep section converted many columns of the data set to numbers, but did not do so with the "always pays" column. The `cmh` function allows you to provide a lambda (or other function) to evaluate whether the value of some column has or does not have the property you are asking questions about. Here, the `pay_λ` function is simple, but the option to provide an encoding function can be used for less obvious situations.

In [12]:
pay_λ=lambda x: 1 if x=="Always" or x=="Often" else 0

print(f"""
others' perception is important → always/often pays on dates, controlling for masculinity self-report
{cmh(d, "perception_importance", "pays_on_dates", "weight", ["masc_self_rate"], dep_c=pay_λ):.1%}

masculinity self-rate → always/often pays on dates, controlling for importance of perception
{cmh(d, "masc_self_rate", "pays_on_dates", "weight", ["perception_importance"], dep_c=pay_λ):.1%}
""")


others' perception is important → always/often pays on dates, controlling for masculinity self-report
100.8%

masculinity self-rate → always/often pays on dates, controlling for importance of perception
118.5%



The survey asked respondents whether being male had any advantages or disadvantages at work. Several options were given, but a large percentage marked _None_ for both questions. Are those who self-report as masculine more likely to feel privileged or picked on at work than other men? The results indicate the opposite:

In [13]:
print(f"""
masculinity self-rate → Being male has advantages at work
{cmh(d, "masc_self_rate", "advantage_at_work", "weight", []):.1%}

masculinity self-rate → Being male has disadvantages at work
{cmh(d, "masc_self_rate", "disadvantage_at_work", "weight", []):.1%}
""")


masculinity self-rate → Being male has advantages at work
86.1%

masculinity self-rate → Being male has disadvantages at work
93.5%



There are a lot more questions you could ask of the data, by modifying the `cmh` calls above. Please do clone the repository and modify the code to ask more questions, add more controls, check reliability via bootstrap, or compare the results to your favorite alternative measures of controlled relationships.